# Question Answering using Embeddings

Many use cases require GPT-3 to respond to user questions with insightful answers. For example, a customer support chatbot may need to provide answers to common questions. The GPT models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

In this notebook we will demonstrate a method for enabling GPT-3 to answer questions using a library of text as a reference, by using document embeddings and retrieval. We'll be using a dataset of Wikipedia articles about the Nusring Skills.

In [19]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
import time
import os

openai.api_key = os.getenv('OPEN_AI_KEY') #Add your API key here

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [20]:
prompt = "Who won the 2020 Summer Olympics men's high jump?"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Marcelo Chierighini of Brazil won the gold medal in the men's high jump at the 2020 Summer Olympics."

In [21]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Sorry, I don't know."

In [22]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places 
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics. 
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and 
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992).

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Gianmarco Tamberi and Mutaz Essa Barshim emerged as joint winners of the event.'

In [23]:
# We have hosted the processed dataset, so you can download it directly without having to recreate it.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

df = pd.read_csv('finalwithtokens.csv')
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

335 rows in the data.


content  \
title                                              heading                                                                                                 
CHAPTER 13 MUSCULOSKELETAL ASSESSMENT 13.3 Comm... Foot Drop Foot drop is the inability to raise t...  CHAPTER 13 MUSCULOSKELETAL ASSESSMENT 13.3 Com...   
CHAPTER 19 SPECIMEN COLLECTION 19.2 Blood Gluco... Glucometer Use It is typically the responsibili...  CHAPTER 19 SPECIMEN COLLECTION 19.2 Blood Gluc...   
CHAPTER 17 ENTERAL TUBE MANAGEMENT 17.3 Assessm... Objective Assessment Objective assessments for ...  CHAPTER 17 ENTERAL TUBE MANAGEMENT 17.3 Assess...   
CHAPTER 5 MATH CALCULATIONS 5.1 Math Calculatio... Learning Objectives â¢  Accurately perform cal...  CHAPTER 5 MATH CALCULATIONS 5.1 Math Calculati...   
CHAPTER 17 ENTERAL TUBE MANAGEMENT 17.2 Basic C... Tube Feeding Enteral Nutrition (EN) refers to n...  CHAPTER 17 ENTERAL TUBE MANAGEMENT 17.2 Basic ...   

                                                                                                       tokens  
title                                              heading                                                     
CHAPTER 13 MUSCULOSKELETAL ASSESSMENT 13.3 Comm... Foot Drop Foot drop is the inability to raise t...     241  
CHAPTER 19 SPECIMEN COLLECTION 19.2 Blood Gluco... Glucometer Use It is typically the responsibili...     630  
CHAPTER 17 ENTERAL TUBE MANAGEMENT 17.3 Assessm... Objective Assessment Objective assessments for ...     263  
CHAPTER 5 MATH CALCULATIONS 5.1 Math Calculatio... Learning Objectives â¢  Accurately perform cal...     367  
CHAPTER 17 ENTERAL TUBE MANAGEMENT 17.2 Basic C... Tube Feeding Enteral Nutrition (EN) refers to n...    1667

In [24]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    time.sleep(7)
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [25]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

Again, we have hosted the embeddings for you so you don't have to re-calculate them from scratch.

In [26]:
#document_embeddings = load_embeddings("embeddings.csv")

# ===== OR, uncomment the below line to recaculate the embeddings from scratch. ========

document_embeddings = compute_doc_embeddings(df)
embeddings_df = pd.DataFrame(document_embeddings)
embeddings_df.to_csv('embeddings.csv', index=False)

In [27]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

('nan Preface', 'Usage Survey and Feedback We would love to hear if you have integrated some or all of this resource\ninto your course. Please use this short survey to report student usage\ninformation every semester for grant reporting purposes to the\nDepartment of Education.  Please use this survey to provide constructive') : [-0.017130045220255852, 0.0021634537260979414, 0.0034936275333166122, -0.018987849354743958, -0.004282511305063963]... (1536 entries)


In [28]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [29]:
order_document_sections_by_query_similarity("What is cultural safety and why is it important when initiating patient interaction in a healthcare setting?", document_embeddings)[:5]

[(0.9328783661893489,
  ('CHAPTER 1 GENERAL SURVEY 1.2 Initiating Patient Interaction',
   'Cultural Safety When initiating patient interaction, it is important to establish cultural\nsafety. Cultural safety refers to the creation of safe spaces for patients to\ninteract with health professionals without judgment or discrimination. See\nand all patients bring a cultural context to interactions in a health care\nsetting is helpful when creating cultural safe spaces. If you discover you\nneed more information about a patientâ\x80\x99s cultural beliefs to tailor your care,\nuse an open-ended question that allows the patient to share what they\nbelieve to be important. For example, you may ask, â\x80\x9cI am interested in your\ncultural background as it relates to your health. Can you share with me\nwhat is important about your cultural background that will help me care\nfor you?â\x80\x9d')),
 (0.8880597483350503,
  ('CHAPTER 2 HEALTH HISTORY 2.2 Health History Basic Concepts',
   'Cultura

In [30]:
order_document_sections_by_query_similarity("What types of open-ended questions can healthcare professionals use to gain a better understanding of a patient's cultural background and tailor their care accordingly?", document_embeddings)[:5]

[(0.8562869672618657,
  ('CHAPTER 2 HEALTH HISTORY 2.2 Health History Basic Concepts',
   'Cultural Safety It is important to conduct a health history in a culturally safe manner.\nCultural safety refers to the creation of safe spaces for patients to interact\nwith health professionals without judgment or discrimination. Focus on\nfactors related to a personâ\x80\x99s cultural background that may influence their\nhealth status. It is helpful to use an open-ended question to allow the\npatient to share what they believe to be important. For example, ask â\x80\x9cI am\ninterested in your cultural background as it relates to your health. Can you\nshare with me what is important to know about your cultural background\nas part of your health care?â\x80\x9d\nIf a patientâ\x80\x99s primary language is not English, it is important to obtain a\nmedical translator, as needed, prior to initiating the health history. The\npatientâ\x80\x99s family member or care partner should not interpret for the

In [31]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [32]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [33]:
prompt = construct_prompt(
    "How can healthcare professionals create safe spaces for patients to interact without judgment or discrimination, and what role do cultural beliefs play in this process?",
    document_embeddings,
    df
)

print("===\n", prompt)

Selected 2 document sections:
('CHAPTER 1 GENERAL SURVEY 1.2 Initiating Patient Interaction', 'Cultural Safety When initiating patient interaction, it is important to establish cultural\nsafety. Cultural safety refers to the creation of safe spaces for patients to\ninteract with health professionals without judgment or discrimination. See\nand all patients bring a cultural context to interactions in a health care\nsetting is helpful when creating cultural safe spaces. If you discover you\nneed more information about a patientâ\x80\x99s cultural beliefs to tailor your care,\nuse an open-ended question that allows the patient to share what they\nbelieve to be important. For example, you may ask, â\x80\x9cI am interested in your\ncultural background as it relates to your health. Can you share with me\nwhat is important about your cultural background that will help me care\nfor you?â\x80\x9d')
('CHAPTER 2 HEALTH HISTORY 2.2 Health History Basic Concepts', 'Cultural Safety It is important t

In [34]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [35]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [36]:
answer_query_with_context("How can healthcare professionals create safe spaces for patients to interact without judgment or discrimination, and what role do cultural beliefs play in this process?", df, document_embeddings)

Selected 2 document sections:
('CHAPTER 1 GENERAL SURVEY 1.2 Initiating Patient Interaction', 'Cultural Safety When initiating patient interaction, it is important to establish cultural\nsafety. Cultural safety refers to the creation of safe spaces for patients to\ninteract with health professionals without judgment or discrimination. See\nand all patients bring a cultural context to interactions in a health care\nsetting is helpful when creating cultural safe spaces. If you discover you\nneed more information about a patientâ\x80\x99s cultural beliefs to tailor your care,\nuse an open-ended question that allows the patient to share what they\nbelieve to be important. For example, you may ask, â\x80\x9cI am interested in your\ncultural background as it relates to your health. Can you share with me\nwhat is important about your cultural background that will help me care\nfor you?â\x80\x9d')
('CHAPTER 2 HEALTH HISTORY 2.2 Health History Basic Concepts', 'Cultural Safety It is important t

'Healthcare professionals can create safe spaces for patients to interact without judgment or discrimination by focusing on factors related to a person\'s cultural background that may influence their health status. It is helpful to use an open-ended question to allow the patient to share what they believe to be important. For example, asking "I am interested in your cultural background as it relates to your health. Can you share with me what is important to know about your cultural background as part of your health care?" This allows the patient to share their cultural beliefs and helps to create a safe space for them to interact with healthcare professionals without judgment or discrimination.'

In [37]:
query = "What is secondary care in the medical field and in what types of settings is it typically provided?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('CHAPTER 23 IV THERAPY MANAGEMENT 23.2 IV Therapy Basics', 'Secondary Fluid Infusion Secondary IV fluid administration is usually an intermittent infusion that\ninfuses at regular intervals (e.g., every 8 hours). This form of IV therapy\nusually contains medications that are supplied in a smaller infusion bag\nand mixed with a diluent fluid like saline (e.g., IV antibiotics). Many\ncommon preparations come in 25 to 100 mL bags.\nSecondary IV therapy is often referred to as â\x80\x9cIV piggybackâ\x80\x9d (IVPB)\nmedication because it is attached to the primary bag of intravenous fluids.\nIn this case, the primary line maintains venous access between drug\ndoses.\nIt is important to remember that not all IV solutions are compatible with\nall IV medications. It is vital for the nurse to triple check that the secondary\nmedications/fluids are compatible with primary fluids. If medication and\nfluids are not compatible, a precipitate may form when the fluids m

In [38]:
query = "In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 0 document sections:


Q: In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?
A: I don't know.


In [39]:
query = "What was unusual about the men’s shotput competition?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('CHAPTER 1 GENERAL SURVEY 1.6 Sample Documentation', 'Sample Documentation of Unexpected Findings Mrs. Smith is a 65-year-old patient who appears older than her stated\nage. She appears slightly agitated during the interview. She is oriented to\nperson only and denies pain. She is wearing a heavy winter coat on a\nwarm summer day and has an unclean body odor. She is slow to respond\nto questions and does not follow commands. She seems to neglect the\nuse of her right arm. Her gait is shuffled with stooped posture but has no\nthe underweight category. Her vital signs were 186/55, pulse 102 and\nirregular, respiratory rate 22, temperature 38.1 Celsius, and pulse oximetry\nwas 88%.')

Q: What was unusual about the men’s shotput competition?
A: I don't know.


In [40]:
query = "In the 2020 Summer Olympics, how many silver medals did Italy win?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 0 document sections:


Q: In the 2020 Summer Olympics, how many silver medals did Italy win?
A: I don't know.


Our Q&A model is less prone to hallucinating answers, and has a better sense of what it does or doesn't know. This works when the information isn't contained in the context; when the question is nonsensical; or when the question is theoretically answerable but beyond GPT-3's powers!

In [41]:
query = "What is the total number of medals won by France, multiplied by the number of Taekwondo medals given out to all countries?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('CHAPTER 5 MATH CALCULATIONS 5.7 Conversions', 'Pounds to Kilograms Converting pounds to kilograms is typically memorized as an equivalency,\nbut letâ\x80\x99s practice using the technique of dimensional analysis.\nSample problem: The patient entered their weight as 137 pounds on their\nintake form. Convert the patientâ\x80\x99s weight to kilograms to document it in\nthe electronic medical record. Round your answer to the nearest tenth.\nCalculate using dimensional analysis.\n1.  Start by identifying kg as the goal unit for which you are solving. Then,\nset up the first fraction so that the numerator matches the goal unit\nof kg.  For the denominator, add 2.2 lbs because the knownequivalency is 1 kg is equivalent to 2.2 pounds. Set up the second\nfraction with pounds in the numerator so that pounds will cross out\ndiagonally to eliminate this unit. Then, add the patientâ\x80\x99s known weight\n(137 lb) in the numerator, with 1 in the denominator because t

In [42]:
query = "What is the tallest mountain in the world?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 0 document sections:


Q: What is the tallest mountain in the world?
A: I don't know.


In [43]:
query = "Who won the grimblesplatch competition at the 2020 Summer Olympic games?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('CHAPTER 1 GENERAL SURVEY 1.6 Sample Documentation', 'Sample Documentation of Expected Findings Mrs. Smith is a 65-year-old patient who appears her stated age. She is\ncalm, cooperative, alert, and oriented x 3. She is well-groomed and her\nclothing is clean and appropriate for the weather. Her speech is clear and\nunderstandable, and she follows instructions appropriately.\nMrs. Smith moves all extremities equally bilaterally with good posture. Her\ngait is smooth and she maintains balance without assistance. Her skin is\nwarm and her mucous membranes are moist. She is  5â\x80\x994â\x80\x9d and she weighs\n143 with a BMI of 24 in the normal weight category. Her vital signs were\nBP 120/70, pulse 74 and regular, respiratory rate 14, temperature 36.8\nCelsius, and pulse oximetry was 98% on room air.')
('CHAPTER 6 NEUROLOGICAL ASSESSMENT 6.7 Assessing Motor Strength', 'Hand Grasps To perform a hand grasp test, extend two fingers on both hands toward\nthe pa